In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [2]:
!pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == 

In [5]:
!pip install -q sacrebleu

In [4]:
# imports
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import load_dataset
import torch
import wandb
from kaggle_secrets import UserSecretsClient
import os, wandb

2025-07-13 19:20:33.145748: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752434433.331319      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752434433.380933      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
# connect wandb library for graphs
secret = UserSecretsClient()
os.environ["WANDB_API_KEY"] = secret.get_secret("WANDB_KEY") 

wandb.login()   

wandb: Currently logged in as: 22cs3033 (22cs3033-rgipt-jais) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [8]:
# loading model and tokenizer into the RAM
model_name = "Qwen/Qwen3-0.6B"
model = AutoModelForCausalLM.from_pretrained(model_name, low_cpu_mem_usage=True )
model.config.use_cache = False
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [9]:
# load the model into GPU
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [10]:
# create the response on default pretrained model
prompt = "Dengue cases on the rise in city"
messages = [
    {"role": "system", "content": "You are a translator. who knows marathai and english language very well. Your task is convert given english sentence in marathi."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]


In [11]:
response

'<think>\nOkay, the user wants me to translate the sentence "Dengue cases on the rise in city" into Marathi. Let me start by breaking down the sentence. "Dengue cases" is straightforward. "On the rise" would be "अव्यापन असते" or "अव्यापन वर्तते". "In city" translates to "तेल असते" or "तेल असते". \n\nNow, putting it all together: "Dengue cases on the rise in city" becomes "तेल असते अव्यापन वर्तते तेल". Wait, is that correct? Let me check. The structure should be "Dengue cases are increasing in the city". So maybe "तेल असते अव्यापन वर्तते तेल" makes sense. Alternatively, "Dengue cases are rising in the city" could be "तेल असते अव्यापन वर्तते तेल". Yes, that seems right. I should make sure there are no grammatical errors. The sentence is clear and the translation is accurate. Alright, that should be the answer.\n</think>\n\nतेल असते अव्यापन वर्तते तेल'

In [12]:
# load the english to marathi translation data from huggingface
ds = load_dataset("anujsahani01/English-Marathi")

README.md:   0%|          | 0.00/206 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/621M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/243M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2637962 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/879321 [00:00<?, ? examples/s]

In [13]:
ds

DatasetDict({
    train: Dataset({
        features: ['english', 'marathi'],
        num_rows: 2637962
    })
    test: Dataset({
        features: ['english', 'marathi'],
        num_rows: 879321
    })
})

In [14]:
ds['train'][10]

{'english': 'The funds collected are used for social purposes.',
 'marathi': 'त्या निधीचा वापर सामाजिक कार्यासाठी करतो.'}

In [15]:
# Dataset is too big so slice it for quick experimentation
df_train = ds["train"].select(range(10000))
df_validation = ds["train"].select(range(10000 , 11000))
df_test = ds["test"].select(range(200))

In [16]:
# Qwen3-0.6B is Causal Language Models so we have finetuned it Instruction-output format. (instruction‑tuning)
def add_instruction(example):
    example["instruction"] = "Convert the English text into Marathi language."
    return example


df_train = df_train.map(add_instruction, batched=False)
df_validation = df_validation.map(add_instruction, batched=False)
df_test = df_test.map(add_instruction, batched=False)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [17]:
ds["train"].column_names

['english', 'marathi']

In [18]:
df_train[0]

{'english': 'Next few months are really crucial for us.',
 'marathi': 'पुढील तीन महिने आमच्यासाठी खूप महत्त्वपूर्ण आहेत.',
 'instruction': 'Convert the English text into Marathi language.'}

In [21]:
# Know about how tokenizer works
inputs = tokenizer('Next few months are really crucial for us.','पुढील तीन महिने आमच्यासाठी खूप महत्त्वपूर्ण आहेत.', 'Convert the English text into Marathi language.' )
inputs

{'input_ids': [5847, 2421, 3951, 525, 2167, 16587, 369, 601, 13, 86162, 72653, 149269, 43647, 91811, 14925, 97, 43647, 60096, 91217, 93948, 42311, 101, 34370, 14925, 228, 87244, 146113, 30484, 107, 31411, 116, 31411, 254, 43647, 14925, 244, 12619, 224, 86162, 91217, 93948, 79238, 30484, 97, 30484, 113, 86162, 12619, 224, 44179, 30484, 96, 14925, 228, 93948, 54784, 97, 13], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [12012, 279, 6364, 1467, 1119, 2876, 66531, 4128, 13]}

In [22]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['Next',
 'Ġfew',
 'Ġmonths',
 'Ġare',
 'Ġreally',
 'Ġcrucial',
 'Ġfor',
 'Ġus',
 '.',
 'à¤ª',
 'à¥ģ',
 'à¤¢',
 'à¥Ģ',
 'à¤²',
 'Ġà¤',
 '¤',
 'à¥Ģ',
 'à¤¨',
 'Ġà¤®',
 'à¤¹',
 'à¤¿à¤',
 '¨',
 'à¥ĩ',
 'Ġà¤',
 'Ĩ',
 'à¤®',
 'à¤ļ',
 'à¥įà¤',
 '¯',
 'à¤¾à¤',
 '¸',
 'à¤¾à¤',
 'ł',
 'à¥Ģ',
 'Ġà¤',
 'ĸ',
 'à¥',
 'Ĥ',
 'à¤ª',
 'Ġà¤®',
 'à¤¹',
 'à¤¤',
 'à¥įà¤',
 '¤',
 'à¥įà¤',
 'µ',
 'à¤ª',
 'à¥',
 'Ĥ',
 'à¤°',
 'à¥įà¤',
 '£',
 'Ġà¤',
 'Ĩ',
 'à¤¹',
 'à¥ĩà¤',
 '¤',
 '.']

In [23]:
instruction = "Convert the English text into Marathi language."
src_text     = "Next few months are really crucial for us."
tgt_text     = "पुढील तीन महिने आमच्यासाठी खूप महत्त्वपूर्ण आहेत."

# Prompt the model should *see*
prompt = f"{instruction} English: {src_text} Marathi:"
print(prompt)

inputs  = tokenizer(prompt, return_tensors="pt") 
print('Length of tokenize form of prompt:', inputs['input_ids'].size())


# Every token sequence in a causal‐LM fine‑tuning setup needs a clear “stop” signal so the model knows when it’s done generating. That’s exactly what tokenizer.eos_token_id is for

labels  = tokenizer(tgt_text, add_special_tokens=False).input_ids + [tokenizer.eos_token_id] #Here labes are tokenize form our marathi output

Convert the English text into Marathi language. English: Next few months are really crucial for us. Marathi:
Length of tokenize form of prompt: torch.Size([1, 23])


In [24]:
inputs

{'input_ids': tensor([[12012,   279,  6364,  1467,  1119,  2876, 66531,  4128,    13,  6364,
            25,  9295,  2421,  3951,   525,  2167, 16587,   369,   601,    13,
          2876, 66531,    25]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [25]:
# The length of tokenize input prompt( instrution + context) is 23

In [26]:
tokenizer.eos_token_id

151645

In [27]:
labels 

[86162,
 72653,
 149269,
 43647,
 91811,
 14925,
 97,
 43647,
 60096,
 91217,
 93948,
 42311,
 101,
 34370,
 14925,
 228,
 87244,
 146113,
 30484,
 107,
 31411,
 116,
 31411,
 254,
 43647,
 14925,
 244,
 12619,
 224,
 86162,
 91217,
 93948,
 79238,
 30484,
 97,
 30484,
 113,
 86162,
 12619,
 224,
 44179,
 30484,
 96,
 14925,
 228,
 93948,
 54784,
 97,
 13,
 151645]

In [28]:
len(labels) # 49 orignal + 1 tokenizer.eos_token_id = 50

50

In [29]:
# For a causal model, you feed it the entire sequence (prompt and target), because it learns by shifting that sequence one token at a time.
model_inputs = tokenizer(prompt + " " + tgt_text + tokenizer.eos_token,  return_tensors="pt")
print('Length of tokenize form of model_inputs:', model_inputs['input_ids'].size())  

# You need to know how many tokens correspond just to the instruction+source (the “prompt”) so that you can later mask them out in the labels.
prompt_len = len(tokenizer(prompt, add_special_tokens=False).input_ids)
print("Tokenized Prompt length:" , prompt_len)

#  Clone so you don’t overwrite the original inputs
labels = model_inputs["input_ids"].clone()


# For every position in the prompt, set label to -100  
# In Hugging Face Trainer, any label token = –100 is ignored by the loss function.
labels[:, :prompt_len] = -100        # works for a batch of size 1 or N
# print('Length that going to be masked' , len(labels))
model_inputs["labels"] = labels

Length of tokenize form of model_inputs: torch.Size([1, 73])
Tokenized Prompt length: 23


In [30]:
labels

tensor([[  -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,  83636,  72653, 149269,  43647,
          91811,  14925,     97,  43647,  60096,  91217,  93948,  42311,    101,
          34370,  14925,    228,  87244, 146113,  30484,    107,  31411,    116,
          31411,    254,  43647,  14925,    244,  12619,    224,  86162,  91217,
          93948,  79238,  30484,     97,  30484,    113,  86162,  12619,    224,
          44179,  30484,     96,  14925,    228,  93948,  54784,     97,     13,
         151645]])

In [31]:
labels.size()

torch.Size([1, 73])

In [32]:
x = labels[0] 
num_ignored = (x == -100).sum().item()
num_not_ignored = (x != -100).sum().item()
print(num_ignored)
print(num_not_ignored)

23
50


In [33]:
model_inputs

{'input_ids': tensor([[ 12012,    279,   6364,   1467,   1119,   2876,  66531,   4128,     13,
           6364,     25,   9295,   2421,   3951,    525,   2167,  16587,    369,
            601,     13,   2876,  66531,     25,  83636,  72653, 149269,  43647,
          91811,  14925,     97,  43647,  60096,  91217,  93948,  42311,    101,
          34370,  14925,    228,  87244, 146113,  30484,    107,  31411,    116,
          31411,    254,  43647,  14925,    244,  12619,    224,  86162,  91217,
          93948,  79238,  30484,     97,  30484,    113,  86162,  12619,    224,
          44179,  30484,     96,  14925,    228,  93948,  54784,     97,     13,
         151645]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]]), 'labels': tensor([[  -100,   -100,   -100,

In [34]:
model_inputs['input_ids'].size()

torch.Size([1, 73])

In [35]:
# this function convert string into vectors
def tokenize_function(examples):

    batch_input_ids      = []
    batch_attention_mask = []
    batch_labels         = []

    # Iterate over the batch
    for instruction, english, marathi in zip(
            examples["instruction"], examples["english"], examples["marathi"]):

        # 1. Build the prompt shown to the model (not trained on)
        prompt_text = f"{instruction} English: {english} Marathi:"
        prompt_ids  = tokenizer(prompt_text, add_special_tokens=False).input_ids

        # 2. Full text = prompt + target + EOS
        full_text = f"{prompt_text} {marathi}{tokenizer.eos_token}"
        tok = tokenizer(full_text, add_special_tokens=False)

        # 3. Create labels: mask the prompt tokens with –100
        label_ids = [-100] * len(prompt_ids) + tok["input_ids"][len(prompt_ids):]

        # 4. Collect results for this example
        batch_input_ids.append(tok["input_ids"])
        batch_attention_mask.append(tok["attention_mask"])
        batch_labels.append(label_ids)

    return {
        "input_ids":      batch_input_ids,
        "attention_mask": batch_attention_mask,
        "labels":         batch_labels,
    }

In [36]:
train_tokenised_ds = df_train.map(tokenize_function, batched=True , num_proc=4 , remove_columns=df_train.column_names)
test_tokenised_ds = df_test.map(tokenize_function, batched=True , num_proc=4 , remove_columns=df_test.column_names)
validation_tokenised_ds = df_validation.map(tokenize_function, batched=True , num_proc=4, remove_columns=df_validation.column_names )

Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/200 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [37]:
test_tokenised_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 200
})

In [38]:
len(test_tokenised_ds[0]['input_ids'])

90

In [39]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({"pad_token": tokenizer.eos_token})
    model.resize_token_embeddings(len(tokenizer))          # update embeddings
model.config.pad_token_id = tokenizer.pad_token_id

In [40]:
for split in (train_tokenised_ds, test_tokenised_ds, validation_tokenised_ds):
    split.set_format(type="torch")

In [41]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    label_pad_token_id=-100,        # keep –100 so they’re ignored in the loss
    pad_to_multiple_of=8,           # speed-up on tensor-core GPUs; omit if CPU
)


In [42]:
train_tokenised_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 10000
})

In [43]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running on", device)

Running on cuda


In [44]:
os.environ["WANDB_PROJECT"] = "qwen_marathi"

In [45]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="qwen_marathi",
    overwrite_output_dir=True,
    
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=5,  
    eval_steps=100,
    logging_steps=100,
    save_total_limit=2,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=200,
    # metric_for_best_model="eval_loss",
    lr_scheduler_type="cosine", # linear or cosine or cosine_with_restarts
    fp16= True,

    #new
    optim="adafactor",  # or adamw_torch
    report_to="wandb",
    run_name="qwen-marathi-run2",
    save_only_model=True,        # HF ≥ 4.38
    save_safetensors=True, 
    #new
    gradient_checkpointing=True,
    # bf16=True,                # TPU v3/v4 natively run in bfloat16
    # tpu_num_cores=8, 
)


In [46]:
from transformers import Trainer
model.to(device) 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenised_ds,
    eval_dataset=validation_tokenised_ds,
    processing_class=tokenizer,        # <-- pass tokenizer here
    data_collator=data_collator,
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:741: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Step,Training Loss
100,1.484300
200,1.378200
300,1.279700
400,1.124600
500,1.011300


TrainOutput(global_step=500, training_loss=1.2556070861816406, metrics={'train_runtime': 1573.1979, 'train_samples_per_second': 6.356, 'train_steps_per_second': 0.318, 'total_flos': 4067213061390336.0, 'train_loss': 1.2556070861816406, 'epoch': 1.0})

In [47]:
trainer.save_model("qwen_marathi/best_model")


In [48]:
df_test[0]

{'english': 'This name is derived from two Sanskrit words Knti and Pur.',
 'marathi': 'हे नाव दोन संस्कृत शब्दांकडून प्राप्त झाले आहे - कांती आणि पुरा.',
 'instruction': 'Convert the English text into Marathi language.'}

In [50]:
# ------------------------------------------------------------
#  Evaluate the fine-tuned model on the held-out test split
#  and report SacreBLEU.  (pip install evaluate tqdm if needed)
# ------------------------------------------------------------
import torch, evaluate
from tqdm.auto import tqdm

metric = evaluate.load("sacrebleu")        # BLEU implementation recommended by HF

model.eval()                               # inference mode – disables dropout, etc.
batch_size         = 8                     # tweak to fit your GPU / CPU
predictions, refs  = [], []

for i in tqdm(range(0, len(df_test), batch_size)):
    batch            = df_test[i : i + batch_size]
    src_sentences    = batch["english"]
    ref_sentences    = batch["marathi"]

    # Build the same prompt format the model saw during training
    prompts = [
        f"Convert the English text into Marathi language. English: {src} Marathi:"
        for src in src_sentences
    ]

    # Tokenise & send to device
    inputs = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512,          # truncate over-long inputs, if any
    ).to(model.device)

    with torch.no_grad():
        generated = model.generate(
            **inputs,
            max_new_tokens=512,  # generation budget
            do_sample=False,     # greedy decoding
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
        )

    # Strip the prompt part from each generated sequence
    input_lens = (inputs.input_ids != tokenizer.pad_token_id).sum(1)
    for gen_ids, in_len in zip(generated, input_lens):
        gen_only_ids = gen_ids[in_len:]                   # remove prompt tokens
        pred_text     = tokenizer.decode(
            gen_only_ids, skip_special_tokens=True
        ).strip()
        predictions.append(pred_text)

    # Each reference must itself be a list (to support multi-ref BLEU)
    refs.extend([[t] for t in ref_sentences])

# ----  Compute BLEU  -------------------------------------------------
bleu_result = metric.compute(predictions=predictions, references=refs)

print(f"SacreBLEU on test set: {bleu_result['score']:.2f}")

  0%|          | 0/25 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
The following generation 

SacreBLEU on test set: 1.89


In [51]:
# 5️⃣ Evaluate on validation and test sets
val_metrics  = trainer.evaluate(eval_dataset=validation_tokenised_ds)
test_metrics = trainer.evaluate(eval_dataset=test_tokenised_ds)

print("Validation metrics:", val_metrics)
print("Test metrics:", test_metrics)

# 6️⃣ Save final tokenizer + config (for inference)
tokenizer.save_pretrained("qwen_marathi/final_tokenizer")
model.save_pretrained("qwen_marathi/final_model")

Validation metrics: {'eval_loss': 1.0184826850891113, 'eval_runtime': 38.7886, 'eval_samples_per_second': 25.781, 'eval_steps_per_second': 6.445, 'epoch': 1.0}
Test metrics: {'eval_loss': 1.0316262245178223, 'eval_runtime': 7.1915, 'eval_samples_per_second': 27.811, 'eval_steps_per_second': 6.953, 'epoch': 1.0}


In [52]:
def translate_to_marathi(english_sentence: str) -> str:
    instruction = "Convert the English text into Marathi language."
    prompt = f"{instruction} English: {english_sentence} Marathi:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate with a reasonable max length
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=False,           # greedy
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )
    # Strip off prompt tokens and decode rest
    generated = outputs[0][inputs.input_ids.size(1):]
    return tokenizer.decode(generated, skip_special_tokens=True)

# Example
print(translate_to_marathi("The objective of this article and the following two articles is to deepen our appreciation for Gods qualities that we may think of less often than his principal attributes."))

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


 या आणि नंतर दोन लेखांच्या विशिष्ट विषयांमध्ये आपल्या देवाच्या विषयांची आपल्या विशिष्ट विषयांची विशिष्ट विषय वाढवण्याची आहे.


In [53]:
df_test[15]

{'english': 'The objective of this article and the following two articles is to deepen our appreciation for Gods qualities that we may think of less often than his principal attributes.',
 'marathi': 'त्याच प्रकारे, आपण जेव्हा यहोवाच्या एखाद्या गुणाची ओळख करून घेतो, त्या गुणावर मनन करतो आणि आपल्या स्वतःच्या जीवनात तो गुण दाखवतो तेव्हा यहोवाच्या त्या गुणाबद्दलची आपली कदर आणखी वाढते.',
 'instruction': 'Convert the English text into Marathi language.'}